In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from IPython.display import display

# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline

# from pandas import HDFStore, read_hdf
# from pathlib import Path

import pickle
import requests, json

# from tabulate import tabulate

pd.options.display.float_format = '{:,.3f}'.format
pd.set_option('display.max_columns', 500)

from catboost import CatBoostRegressor



import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    from sklearn import preprocessing

In [2]:
data_dir = './dataPool/'
csv_file = '2_ClaimLoss.csv'

read_csv = data_dir + csv_file

In [3]:
df_group_raw = pd.read_csv(read_csv, encoding ='utf_8')
df_group_raw

,Loss$,Group,CnaeSession,Uf,FundationDate,Equity,RefYearMonth,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,0,G1,"ARTES, CULTURA, ESPORTE E RECREAÇÃO",MT,2016,7320000,201804,nan,nan,nan,nan
1,0,G5,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES...",MT,1993,7320000,201708,nan,nan,nan,nan
2,0,G1,ATIVIDADES IMOBILIÁRIAS,DF,1990,2320000,201806,nan,nan,nan,nan
3,"1,000,00",G1,"ATIVIDADES PROFISSIONAIS, CIENTÍFICAS E TÉCNICAS",RR,1998,5320000,201806,nan,nan,nan,nan
4,"500,00",G4,SERVIÇOS DOMÉSTICOS,TO,2016,7320000,201801,nan,nan,nan,nan
5,"2,000,00",G2,SERVIÇOS DOMÉSTICOS,MS,1987,5320000,201806,nan,nan,nan,nan
6,0,G4,"ADMINISTRAÇÃO PÚBLICA, DEFESA E SEGURIDADE SOCIAL",AL,1998,500000,201805,nan,nan,nan,nan
7,"20,000,00",G1,ALOJAMENTO E ALIMENTAÇÃO,SE,1998,5320000,201803,nan,nan,nan,nan
8,0,G3,SERVIÇOS DOMÉSTICOS,SE,1998,5320000,201709,nan,nan,nan,nan
9,"1,000,00",G2,"ARTES, CULTURA, ESPORTE E RECREAÇÃO",PR,2001,15320000,201712,nan,nan,nan,nan


In [4]:
df_group_raw = df_group_raw.iloc[:, :7]

display (df_group_raw.head(3))
print(df_group_raw.shape)

,Loss$,Group,CnaeSession,Uf,FundationDate,Equity,RefYearMonth
0,0,G1,"ARTES, CULTURA, ESPORTE E RECREAÇÃO",MT,2016,7320000,201804
1,0,G5,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES...",MT,1993,7320000,201708
2,0,G1,ATIVIDADES IMOBILIÁRIAS,DF,1990,2320000,201806


(9999, 7)


In [5]:
for i in list(df_group_raw):
    print(i)
    print(df_group_raw[i].value_counts())

Loss$
0            6475
1,000,00      740
10,000,00     714
500,00        697
2,000,00      696
20,000,00     677
Name: Loss$, dtype: int64
Group
G3    2040
G2    2021
G5    1989
G1    1980
G4    1969
Name: Group, dtype: int64
CnaeSession
COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E MOTOCICLETAS           521
INFORMAÇÃO E COMUNICAÇÃO                                             513
EDUCAÇÃO                                                             513
ATIVIDADES IMOBILIÁRIAS                                              501
ORGANISMOS INTERNACIONAIS E OUTRAS INSTITUIÇÕES EXTRATERRITORIAIS    497
AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PESCA E AQÜICULTURA       484
CONSTRUÇÃO                                                           481
ÁGUA, ESGOTO, ATIVIDADES DE GESTÃO DE RESÍDUOS E DESCONTAMINAÇÃO     481
ARTES, CULTURA, ESPORTE E RECREAÇÃO                                  477
ALOJAMENTO E ALIMENTAÇÃO                                             473
INDÚSTRIAS DE TRANSFORMAÇÃO    

In [6]:
df = df_group_raw.iloc[:, :6]
df

,Loss$,Group,CnaeSession,Uf,FundationDate,Equity
0,0,G1,"ARTES, CULTURA, ESPORTE E RECREAÇÃO",MT,2016,7320000
1,0,G5,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES...",MT,1993,7320000
2,0,G1,ATIVIDADES IMOBILIÁRIAS,DF,1990,2320000
3,"1,000,00",G1,"ATIVIDADES PROFISSIONAIS, CIENTÍFICAS E TÉCNICAS",RR,1998,5320000
4,"500,00",G4,SERVIÇOS DOMÉSTICOS,TO,2016,7320000
5,"2,000,00",G2,SERVIÇOS DOMÉSTICOS,MS,1987,5320000
6,0,G4,"ADMINISTRAÇÃO PÚBLICA, DEFESA E SEGURIDADE SOCIAL",AL,1998,500000
7,"20,000,00",G1,ALOJAMENTO E ALIMENTAÇÃO,SE,1998,5320000
8,0,G3,SERVIÇOS DOMÉSTICOS,SE,1998,5320000
9,"1,000,00",G2,"ARTES, CULTURA, ESPORTE E RECREAÇÃO",PR,2001,15320000


## Features and label

In [7]:
df_data = df.drop(columns=['Loss$'])
df_label = df['Loss$'].str.replace(',', '').astype(float)

In [8]:
display (df_data.head(3))
display (df_label.head(3))

,Group,CnaeSession,Uf,FundationDate,Equity
0,G1,"ARTES, CULTURA, ESPORTE E RECREAÇÃO",MT,2016,7320000
1,G5,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES...",MT,1993,7320000
2,G1,ATIVIDADES IMOBILIÁRIAS,DF,1990,2320000


0   0.000
1   0.000
2   0.000
Name: Loss$, dtype: float64

## (Optional) One-hot encoding (No need for our current project)

In [9]:
from sklearn import preprocessing
from collections import defaultdict
d = defaultdict(preprocessing.LabelEncoder)
d

defaultdict(sklearn.preprocessing.label.LabelEncoder, {})

### Assign encoder

In [10]:
df_dataEncoder = df.drop(columns=['Group','Equity'])[:4]
df_dataEncoder

,Loss$,CnaeSession,Uf,FundationDate
0,0,"ARTES, CULTURA, ESPORTE E RECREAÇÃO",MT,2016
1,0,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES...",MT,1993
2,0,ATIVIDADES IMOBILIÁRIAS,DF,1990
3,"1,000,00","ATIVIDADES PROFISSIONAIS, CIENTÍFICAS E TÉCNICAS",RR,1998


In [11]:
df_encoded = df_dataEncoder.apply(lambda x: d[x.name].fit_transform(x))
display(df_encoded)
print(d)
print(d['CnaeSession'])

,Loss$,CnaeSession,Uf,FundationDate
0,0,1,1,3
1,0,0,1,1
2,0,2,0,0
3,1,3,2,2


defaultdict(<class 'sklearn.preprocessing.label.LabelEncoder'>, {'Loss$': LabelEncoder(), 'CnaeSession': LabelEncoder(), 'Uf': LabelEncoder(), 'FundationDate': LabelEncoder()})
LabelEncoder()


In [12]:
df_encoded_rev = df_encoded.apply(lambda x: d[x.name].inverse_transform(x))
display(df_encoded_rev)

,Loss$,CnaeSession,Uf,FundationDate
0,0,"ARTES, CULTURA, ESPORTE E RECREAÇÃO",MT,2016
1,0,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES...",MT,1993
2,0,ATIVIDADES IMOBILIÁRIAS,DF,1990
3,"1,000,00","ATIVIDADES PROFISSIONAIS, CIENTÍFICAS E TÉCNICAS",RR,1998


### Feature mapping

In [13]:
class feature_mapping(object):
    def __init__(self, d, curr_feat):
        self.d = d
        self.curr_feat = curr_feat
    def getMapping(self):
        mapping_all = dict()
        for one_feat in self.curr_feat:
            mapping_all[one_feat] = dict(zip(d[one_feat].classes_, range(len(d[one_feat].classes_))))
#             mapping_all.append(dict(zip(d[one_feat].classes_, 
#                                     range(len(d[one_feat].classes_)))))          
        return mapping_all
       

In [14]:
myMap = feature_mapping(d, list(df_encoded))
display(myMap.getMapping())
print(myMap.getMapping()['CnaeSession'])
print({k:v for k, v in myMap.getMapping()['CnaeSession'].items() if v == 0})


{'Loss$': {'0': 0, '1,000,00': 1},
 'CnaeSession': {'AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PESCA E AQÜICULTURA': 0,
  'ARTES, CULTURA, ESPORTE E RECREAÇÃO': 1,
  'ATIVIDADES IMOBILIÁRIAS': 2,
  'ATIVIDADES PROFISSIONAIS, CIENTÍFICAS E TÉCNICAS': 3},
 'Uf': {'DF': 0, 'MT': 1, 'RR': 2},
 'FundationDate': {1990: 0, 1993: 1, 1998: 2, 2016: 3}}

{'AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PESCA E AQÜICULTURA': 0, 'ARTES, CULTURA, ESPORTE E RECREAÇÃO': 1, 'ATIVIDADES IMOBILIÁRIAS': 2, 'ATIVIDADES PROFISSIONAIS, CIENTÍFICAS E TÉCNICAS': 3}
{'AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PESCA E AQÜICULTURA': 0}


### End of optional one-hot encoding for feature

## One-hot encoding for label

In [15]:
# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()
# le.fit(df_label_cat)
# df_label = le.transform(df_label_cat) 

# ## demo
# print (list(le.classes_))
# print(list(df_label)[:10])
# print(le.inverse_transform([0, 0, 1, 2]))

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_data, df_label, test_size=0.33, random_state=42)

In [17]:
display (X_train.head(3))
display (X_test.head(3))
display (y_train[:3])
display (y_test[:3])

,Group,CnaeSession,Uf,FundationDate,Equity
5756,G4,CONSTRUÇÃO,PI,2010,5320000
5026,G3,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,SC,1996,1000000
9233,G3,CONSTRUÇÃO,CE,2001,5320000


,Group,CnaeSession,Uf,FundationDate,Equity
5344,G3,OUTRAS ATIVIDADES DE SERVIÇOS,RN,1997,1000000
7444,G2,"ARTES, CULTURA, ESPORTE E RECREAÇÃO",MT,1996,15320000
1731,G4,INFORMAÇÃO E COMUNICAÇÃO,MT,1987,1500000


5756        0.000
5026   50,000.000
9233        0.000
Name: Loss$, dtype: float64

5344     200,000.000
7444     200,000.000
1731   2,000,000.000
Name: Loss$, dtype: float64

In [18]:
# specify the training parameters 
model2 = CatBoostRegressor(iterations=10, depth=6, learning_rate=1, 
                           loss_function='RMSE', 
                           custom_metric='AUC', logging_level='Verbose')
#train the model
cat_features_index = [0,1,2,3]
model2.fit(X_train, y_train, cat_features = cat_features_index)
# make the prediction using the resulting model
preds_class = model2.predict(X_test)
print("class = ", preds_class)


0:	learn: 543444.1819783	total: 57.2ms	remaining: 515ms
1:	learn: 543234.5997767	total: 62.5ms	remaining: 250ms
2:	learn: 543234.5983471	total: 69.2ms	remaining: 162ms
3:	learn: 542794.5245006	total: 83.6ms	remaining: 125ms
4:	learn: 541585.5088973	total: 103ms	remaining: 103ms
5:	learn: 541333.6934643	total: 117ms	remaining: 77.9ms
6:	learn: 540974.2206322	total: 130ms	remaining: 55.7ms
7:	learn: 540911.0070657	total: 140ms	remaining: 34.9ms
8:	learn: 540382.6787116	total: 153ms	remaining: 17ms
9:	learn: 539472.1850215	total: 167ms	remaining: 0us
class =  [247952.91903296 220067.9273441  288204.01177315 ... 220067.9273441
 230602.17036942 220067.9273441 ]


In [19]:
print (preds_class[:20])

[247952.91903296 220067.9273441  288204.01177315 231896.86718728
 221065.00524302 220067.9273441  209901.55989487 258487.16205828
 205694.44947635 258487.16205828 235718.31200981 238588.74223138
 277669.76874782 228054.49920606 245227.55650223 234448.30776943
 231896.86718728 209264.25275771 220067.9273441  253953.58325866]


## Model serialization

In [20]:
import pickle
myPickle = 'model2.pkl'
pickle.dump(model2, open(myPickle, 'wb'))

In [21]:
modelCatBoostRegressor = pickle.load(open(myPickle, 'rb'))

In [22]:
modelCatBoostRegressor

## Output prediction with index and group

In [23]:
def getIndexAndGroup(data_input):
    my_predIndex = modelCatBoost.predict(data_input).astype('int').flatten() ## flatten for narray to list
    my_predGroup = encoder.inverse_transform(my_predIndex)
    print(my_predIndex, my_predGroup)
    return my_predIndex, my_predGroup

## Input data for prediction

### From test dataset

In [24]:
myInput = X_test[:10]
display(myInput)

,Group,CnaeSession,Uf,FundationDate,Equity
5344,G3,OUTRAS ATIVIDADES DE SERVIÇOS,RN,1997,1000000
7444,G2,"ARTES, CULTURA, ESPORTE E RECREAÇÃO",MT,1996,15320000
1731,G4,INFORMAÇÃO E COMUNICAÇÃO,MT,1987,1500000
8719,G2,OUTRAS ATIVIDADES DE SERVIÇOS,ES,2016,2320000
4521,G3,"ÁGUA, ESGOTO, ATIVIDADES DE GESTÃO DE RESÍDUOS...",SC,1998,1000000
7453,G4,EDUCAÇÃO,ES,2015,5320000
576,G4,CONSTRUÇÃO,RS,1998,7320000
7428,G3,INDÚSTRIAS EXTRATIVAS,PA,1990,1000000
5577,G4,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES...",MS,2016,5320000
439,G2,CONSTRUÇÃO,AC,1992,1000000


In [25]:
modelCatBoostRegressor.predict(myInput)

array([247952.91903296, 220067.9273441 , 288204.01177315, 231896.86718728,
       221065.00524302, 220067.9273441 , 209901.55989487, 258487.16205828,
       205694.44947635, 258487.16205828])

In [26]:
print(y_test[:3])

5344     200,000.000
7444     200,000.000
1731   2,000,000.000
Name: Loss$, dtype: float64


### From manually input

In [27]:
dataInput = {'Group': 'G4',
             'CnaeSession':'INFORMAÇÃO E COMUNICAÇÃO', 
             'Uf':'MT', 
             'FundationDate':'1987',
             'Equity':1500000,
            }

correcFeatOrder = ['Group', 'CnaeSession', 'Uf', 'FundationDate', 'Equity']
df_dataInput = pd.DataFrame.from_dict([dataInput])[correcFeatOrder]

display(df_dataInput)

,Group,CnaeSession,Uf,FundationDate,Equity
0,G4,INFORMAÇÃO E COMUNICAÇÃO,MT,1987,1500000


In [28]:
modelCatBoostRegressor.predict(df_dataInput)

array([288204.01177315])

### From flask

In [30]:
url = "http://localhost:8000/model2"
data = json.dumps(dataInput)
data
r = requests.post(url, data)
r
print(r.json())

{'predLoss': [288204.0117731468]}


### From heroku

In [ ]:
# urlnew = "https://predgroup.herokuapp.com/model2"
# data = json.dumps(dataInput)
# r = requests.post(urlnew, data)

# print(r.json())

### (Optional) Feature Importances

In [ ]:
# print (classification_report(y_test, my_loaded_model.predict(X_test)))
# print (model.feature_importances_)
# col = list(X_train)
# df = pd.DataFrame({'imp': model.feature_importances_, 'col':col})
# df = df.sort_values(['imp','col'], ascending=[True, False])
# _ = df.plot(kind='barh', x='col', y='imp', figsize=(8,6))

In [ ]:
import pandas as pd
import requests, json
def getIndexAndGroup(data_input):
    my_predIndex = modelCatBoost.predict(data_input).astype('int').flatten() ## flatten for narray to list
    my_predGroup = encoder.inverse_transform(my_predIndex)
    print(my_predIndex, my_predGroup)
    return my_predIndex, my_predGroup
dataInput = {'CnaeSession':'ALOJAMENTO E ALIMENTAÇÃO', 
             'Uf':'RO', 
             'FundationDate':'1997',
             'Equity':15320000,
            }
correcFeatOrder = ['CnaeSession', 'Uf', 'FundationDate', 'Equity']
df_dataInput = pd.DataFrame.from_dict([dataInput])[correcFeatOrder]
print(df_dataInput)

url = "https://predgroup.herokuapp.com/"
data = json.dumps(dataInput)
r = requests.post(url, data)
print(r.json()) 